In [ ]:
import calcbench as cb
import pandas as pd
import requests

In [ ]:
spy_holdings_url = "https://us.spdrs.com/site-content/xls/SPY_All_Holdings.xls?fund=SPY&docname=All+Holdings&onyx_code1=&onyx_code2="

In [ ]:
pd.read_csv?

In [ ]:
QQQ_holdings = pd.read_csv('./etf-constituents-06-02-2022 QQQ.csv', header=0) # From https://www.barchart.com/etfs-funds/quotes/QQQ/constituents
QQQ_holdings = QQQ_holdings[:-1]
SPY_holdings = pd.read_csv('./etf-constituents-06-02-2022 SPY.csv', header=0) # From https://www.barchart.com/etfs-funds/quotes/SPY/constituents
SPY_holdings = SPY_holdings[:-1]

In [ ]:
def get_holdings(file_name: str):
    holdings = pd.read_csv(file_name, header=0) # From https://www.barchart.com/etfs-funds/quotes/QQQ/constituents
    holdings = holdings[:-1]
    holdings.Symbol = holdings.Symbol.replace({'GOOGL' : "GOOG", "BRK.B" : "BRK"})
    holdings = holdings.set_index('Symbol')
    holdings = holdings['% Holding'].str.rstrip('%').astype('float')
    holdings = holdings[holdings.index.dropna()]
    return holdings

In [ ]:
QQQ_holdings = get_holdings('./etf-constituents-06-02-2022 QQQ.csv')
SPY_holdings = get_holdings('./etf-constituents-06-02-2022 SPY.csv')

In [ ]:
SPY_holdings

In [ ]:
SPY_holdings.index.union(QQQ_holdings.index).unique()

In [ ]:
income_statement_metrics = [
    "Revenue",
    "CostOfRevenue",
    "GrossProfit",
    "SGAExpense",
    "OperatingExpenses",
    "OperatingIncome",
    "EBIT",
    "InterestExpense",
    "IncomeTaxes",
    "NetIncome",
]

In [ ]:
cb.standardized?

In [ ]:
d = cb.standardized(metrics=income_statement_metrics, company_identifiers=list(SPY_holdings.index.union(QQQ_holdings.index).unique()))

In [ ]:
d

In [ ]:
d = cb.standardized_data(
    company_identifiers=SPY_holdings.index,
    metrics=income_statement_metrics + ["MarketCapAtEndOfPeriod"],
    year=2016,
    period="q1",
)

In [ ]:
percent_of_company_fund_owns = (
    value_of_fund_holdings / d["MarketCapAtEndOfPeriod"].loc["2016"]
)

In [ ]:
for metric in income_statement_metrics:
    print(metric, (d.loc["2016"][metric] * percent_of_company_fund_owns).sum())